# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
df=pd.read_csv("weather_data.csv")
df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,rikitea,15.0,PF,1.584896e+09,73.0,-23.12,-134.97,298.80,1.66
1,ishigaki,9.0,JP,1.584896e+09,92.0,24.34,124.16,296.20,7.01
2,clyde river,1.0,CA,1.584896e+09,83.0,70.47,-68.59,250.15,1.50
3,port elizabeth,75.0,ZA,1.584896e+09,88.0,-33.92,25.57,296.15,5.70
4,ushuaia,98.0,AR,1.584896e+09,57.0,-54.80,-68.30,282.15,3.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
locations=df[["Lat","Lng"]]
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=df.Humidity,dissipating=False,max_intensity=100,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
#Convert "max temp" from K to F:
df["Max Temp"]=df["Max Temp"]*1.8-459.67


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
hotel_df=df.loc[(df["Max Temp"]<80) & (df["Cloudiness"]==0) & (df["Wind Speed"]<5)
               & (df["Max Temp"]>60) &(df["Humidity"]<80)]
hotel_df=hotel_df.dropna(how='any')
hotel_df=hotel_df.reset_index(drop=True)
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,hermanus,0.0,ZA,1.584896e+09,78.0,-34.42,19.23,69.998,0.89
1,geraldton,0.0,AU,1.584896e+09,44.0,-28.77,114.60,78.800,3.60
2,rahimatpur,0.0,IN,1.584896e+09,40.0,17.60,74.20,71.150,2.08
3,mount isa,0.0,AU,1.584896e+09,53.0,-20.73,139.50,75.200,4.85
4,waddan,0.0,LY,1.584896e+09,22.0,29.16,16.14,75.650,3.83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
#Add "Hotel Name" column to the DataFrame:
hotel_df["Hotel Name"]=""

In [39]:
lat=hotel_df["Lat"]
lng=hotel_df["Lng"]

In [40]:
#Set base url and parameters:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={
    "radius":5000,
    "type":"hotel",
    "key":g_key}


In [41]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    url=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,'Hotel Name']=url["results"][1]["name"]
    except (KeyError,IndexError):
        print("Missing field/result... skipping.")


In [42]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,hermanus,0.0,ZA,1.584896e+09,78.0,-34.42,19.23,69.998,0.89,Misty Waves Boutique Hotel
1,geraldton,0.0,AU,1.584896e+09,44.0,-28.77,114.60,78.800,3.60,Broadwater Mariner Resort
2,rahimatpur,0.0,IN,1.584896e+09,40.0,17.60,74.20,71.150,2.08,State Bank Of India
3,mount isa,0.0,AU,1.584896e+09,53.0,-20.73,139.50,75.200,4.85,ibis Styles Mt Isa Verona
4,waddan,0.0,LY,1.584896e+09,22.0,29.16,16.14,75.650,3.83,Alternative (Authorized agent for the company'...


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations=hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations,
                                 info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

# Create heat layer
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)


# Display Map

In [45]:
fig

Figure(layout=FigureLayout(height='420px'))